This should be the final pipeline for the cleaning and preparing the dataset. End result is a data frame, ready to be tossed into a ML model.

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

Loading the data, and convert all the strings to lower case for uniformity.

In [2]:
df_0 = pd.read_csv("../data/Training_Set_Values.csv")
df_1 = pd.read_csv("../data/Training_Set_labels.csv")
df = df_0.merge(df_1, on='id')
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].map(lambda x: x.lower() if isinstance(x, str) else x)

Print names of the columns that contain NaNs. Need to fix them.

In [3]:
cols_with_nan = df.columns[df.isna().any()].tolist()
print(cols_with_nan)

['funder', 'installer', 'subvillage', 'public_meeting', 'scheme_management', 'scheme_name', 'permit']


Some columns have no NaNs, but have error in recording, such as latitude and longitude recorded as 0.0000, or population, which contains several 0s, 1s and values smaller than 10.

First, fixing subvillage. This is important, because we use the column 'subvillage' to impute missing values in other columns.

In [4]:
# 1. Replace blank strings with NaN
df['subvillage'] = df['subvillage'].replace('', pd.NA)

# 2. Function to fill NaN with fallback hierarchy
def fill_subvillage(row):
    if pd.isna(row['subvillage']):
        ward_mode = df[df['ward'] == row['ward']]['subvillage'].mode()
        if not ward_mode.empty:
            return ward_mode[0]

        lga_mode = df[df['lga'] == row['lga']]['subvillage'].mode()
        if not lga_mode.empty:
            return lga_mode[0]

        region_mode = df[df['district_code'] == row['district_code']]['subvillage'].mode()
        if not region_mode.empty:
            return region_mode[0]

        return 'unknown'

    return row['subvillage']

# 3. Apply the function
df['subvillage'] = df.apply(fill_subvillage, axis=1)

In [5]:
df['subvillage'].isna().sum()

0

Now, fixing the missing Lat and Long coordinate values

In [6]:
df['latitude'] = df['latitude'].apply(lambda x: pd.NA if abs(x) < 1e-6 else x)
df['longitude'] = df['longitude'].apply(lambda x: pd.NA if abs(x) < 1e-6 else x)

In [7]:
geo_fallback_order = ['subvillage', 'ward', 'lga', 'district_code', 'region_code']

def fill_missing_geo(df, group_cols):
    for col in group_cols:
        # Compute group-wise means (excluding missing lat/lon)
        group_means = df.dropna(subset=['latitude', 'longitude']).groupby(col)[['latitude', 'longitude']].mean()

        def fill(row):
            if pd.isna(row['latitude']) or pd.isna(row['longitude']):
                key = row[col]
                if key in group_means.index:
                    if pd.isna(row['latitude']):
                        row['latitude'] = group_means.loc[key, 'latitude']
                    if pd.isna(row['longitude']):
                        row['longitude'] = group_means.loc[key, 'longitude']
            return row

        df = df.apply(fill, axis=1)

        # Stop early if no more NaNs
        if df['latitude'].isna().sum() == 0 and df['longitude'].isna().sum() == 0:
            break

    return df

df = fill_missing_geo(df, geo_fallback_order)

After fixing the latitude and longitude, lets fix population values. Since these are heavily skewed, use median values to fill instead of mean.


In [8]:
# 1. Convert suspicious population values to NaN
df['population'] = df['population'].apply(lambda x: pd.NA if x in [0, 1] else x)

# 2. Define fallback order from most local to most general
fallback_order = ['subvillage','ward', 'lga', 'district_code', 'region_code']

# 3. Imputation function using median per group
def fill_missing_population(df, levels):
    for level in levels:
        # Calculate median population per group (excluding missing)
        group_medians = df.dropna(subset=['population']).groupby(level)['population'].median()

        def fill(row):
            if pd.isna(row['population']):
                key = row[level]
                if key in group_medians:
                    row['population'] = group_medians.loc[key]
            return row

        df = df.apply(fill, axis=1)

        # Exit early if all values are filled
        if df['population'].isna().sum() == 0:
            break

    return df

# 4. Apply the imputation
df = fill_missing_population(df, fallback_order)

# Optional: Check remaining NaNs
print("Remaining missing population values:", df['population'].isna().sum())

Remaining missing population values: 0


construction_year and gps_height also contain many 0s. Gotta fix those too.

In [9]:
def geo_groupwise_fill(df, target_col, group_cols):
    for col in group_cols:
        group_medians = df.dropna(subset=[target_col]).groupby(col)[target_col].median()

        def fill(row):
            if pd.isna(row[target_col]):
                key = row[col]
                if key in group_medians.index:
                    row[target_col] = group_medians.loc[key]
            return row

        df = df.apply(fill, axis=1)

        if df[target_col].isna().sum() == 0:
            break

    return df


In [ ]:
def geo_groupwise_fill_mode(df, target_col, group_cols):
    for col in group_cols:
        group_medians = df.dropna(subset=[target_col]).groupby(col)[target_col].agg(lambda x: x.mode().iloc[0])

        def fill(row):
            if pd.isna(row[target_col]):
                key = row[col]
                if key in group_medians.index:
                    row[target_col] = group_medians.loc[key]
            return row

        df = df.apply(fill, axis=1)

        if df[target_col].isna().sum() == 0:
            break

    return df


In [22]:
geo_cols = ['subvillage', 'ward', 'lga', 'district_code', 'region_code']
df['gps_height'] = df['gps_height'].apply(lambda x: pd.NA if x in [0] else x)
df['construction_year'] = df['construction_year'].apply(lambda x: pd.NA if x in [0] else x)

df = geo_groupwise_fill(df, 'gps_height', geo_cols)
df = geo_groupwise_fill_mode(df, 'construction_year', geo_cols)


from the exploratory analysis, we know that there are only two NaN values in the column wtp_name. Since there already is label called 'none', we label these two missing values also as none.

In [11]:
df['wpt_name'] = df['wpt_name'].fillna('none')


same thing, different column ('management_scheme'), different label ('other')

In [12]:
df['scheme_management'] = df['scheme_management'].fillna('other')

Columns with High cardinality, lets handle them by keeping the values that together make up the top 50% of the dataset, and label the rest as "other".

In [13]:
def relabel_by_coverage(df, column, threshold=0.5, new_label='other'):
    """
    Keep only the categories in `column` that cumulatively make up `threshold` (e.g., 0.5 for 50%) of rows.
    Others are relabeled as `new_label`.
    """
    # Get value counts and cumulative percentage
    value_counts = df[column].value_counts(normalize=True)
    cumulative = value_counts.cumsum()

    # Categories to keep: top ones covering up to the threshold
    keep_labels = cumulative[cumulative <= threshold].index.tolist()

    # Apply relabeling
    df[column] = df[column].apply(lambda x: x if x in keep_labels else new_label)

    return df


In [14]:
df = relabel_by_coverage(df, 'funder', threshold=0.5)
df = relabel_by_coverage(df, 'installer', threshold=0.5)
df = relabel_by_coverage(df, 'scheme_name', threshold=0.5)


filled in quite some columns. Check again what's missing?

In [17]:
cols_with_nan = df.columns[df.isna().any()].tolist()
print(cols_with_nan)

['public_meeting', 'permit']


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Step 1: Select features
features = ['management', 'scheme_management', 'scheme_name','payment', 'population','basin','funder','installer']
df_model = df[features + ['permit']].copy()

# Step 2: One-hot encode categorical features
df_model = pd.get_dummies(df_model, columns=features, drop_first=True)

# Step 3: Separate known and unknown permit rows
df_known = df_model[df_model['permit'].notna()]
df_unknown = df_model[df_model['permit'].isna()]

# Step 4: Train/test split on known data
X = df_known.drop(columns='permit')
y = df_known['permit'].astype(bool)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y
)

# Step 5: Train model
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Step 6: Evaluate model
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on known data: {accuracy:.4f}")

# Step 7: If accuracy is acceptable, predict missing permit values
if accuracy > 0.8:  # or your own threshold
    X_missing = df_unknown.drop(columns='permit')
    predicted_permit = clf.predict(X_missing)
    df.loc[df['permit'].isna(), 'permit'] = predicted_permit
    print("✅ Missing permit values filled using model.")
else:
    print("⚠️ Accuracy too low. Consider another method or more features.")

Accuracy on known data: 0.9139
✅ Missing permit values filled using model.


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Step 1: Select features
features = ['management', 'scheme_management', 'scheme_name','payment', 'population','basin','funder','installer','permit']
df_model = df[features + ['public_meeting']].copy()

# Step 2: One-hot encode categorical features
df_model = pd.get_dummies(df_model, columns=features, drop_first=True)

# Step 3: Separate known and unknown permit rows
df_known = df_model[df_model['public_meeting'].notna()]
df_unknown = df_model[df_model['public_meeting'].isna()]

# Step 4: Train/test split on known data
X = df_known.drop(columns='public_meeting')
y = df_known['public_meeting'].astype(bool)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y
)

# Step 5: Train model
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Step 6: Evaluate model
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on known data: {accuracy:.4f}")

# Step 7: If accuracy is acceptable, predict missing permit values
if accuracy > 0.8:  # or your own threshold
    X_missing = df_unknown.drop(columns='public_meeting')
    predicted_permit = clf.predict(X_missing)
    df.loc[df['public_meeting'].isna(), 'public_meeting'] = predicted_permit
    print("✅ Missing permit values filled using model.")
else:
    print("⚠️ Accuracy too low. Consider another method or more features.")


Accuracy on known data: 0.9530
✅ Missing permit values filled using model.


OK, this fixes the columns we are interested in.

In [20]:
import geopandas as gpd

# Read the shapefile (Tanzania boundaries)
gdf_tz = gpd.read_file("../data/gadm41_TZA_shp/gadm41_TZA_1.shp")

# Check projection
print(gdf_tz.crs)
gdf_tz.head()


ModuleNotFoundError: No module named 'geopandas'

In [ ]:
from shapely.geometry import Point

# Drop rows with missing coordinates
df_geo = df.dropna(subset=['latitude', 'longitude']).copy()

# Create Point geometry from long/lat
df_geo['geometry'] = df_geo.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

# Convert to GeoDataFrame with WGS84 CRS
gdf_pumps = gpd.GeoDataFrame(df_geo, geometry='geometry', crs="EPSG:4326")


In [ ]:
import folium

# Center the map on Tanzania
center_lat, center_lon = -6.3690, 34.8888
m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

# Add Tanzania region boundaries
folium.GeoJson(gdf_tz, name="Tanzania Regions").add_to(m)

# Define color mapping for status groups
color_map = {
    'functional': 'green',
    'non functional': 'red',
    'functional needs repair': 'orange'
}

# Add water pump markers colored by status
for _, row in gdf_pumps.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=2,
        color=color_map.get(row['status_group'], 'gray'),
        fill=True,
        fill_opacity=0.6,
        popup=folium.Popup(
    f"Pump ID: {row['id']}<br>Region: {row['region']}<br>Status: {row['status_group']}",
    max_width=250
)

    ).add_to(m)

# Optional: add layer control
folium.LayerControl().add_to(m)

# Show map (in Jupyter) or save to file
#m.save("tanzania_water_pumps_map.html")
m


In [23]:
df.to_csv('your_filename.csv', index=False)


In [24]:
pip install geopandas folium shapely

     -------------------------------------- 323.6/323.6 kB 5.0 MB/s eta 0:00:00
     -------------------------------------- 110.9/110.9 kB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 23.1 MB/s eta 0:00:00
     --------------------------------------- 15.9/15.9 MB 34.4 MB/s eta 0:00:00
     --------------------------------------- 16.2/16.2 MB 32.7 MB/s eta 0:00:00
     ---------------------------------------- 6.1/6.1 MB 18.5 MB/s eta 0:00:00
     ---------------------------------------- 88.4/88.4 kB ? eta 0:00:00
  Using cached branca-0.8.1-py3-none-any.whl (26 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
Note: you may need to restart the ke

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
tensorflow-intel 2.11.0 requires keras<2.12,>=2.11.0, but you have keras 2.10.0 which is incompatible.
tensorflow-intel 2.11.0 requires tensorboard<2.12,>=2.11, but you have tensorboard 2.10.1 which is incompatible.
tensorflow-intel 2.11.0 requires tensorflow-estimator<2.12,>=2.11.0, but you have tensorflow-estimator 2.10.0 which is incompatible.
scipy 1.9.1 requires numpy<1.25.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 2.0.2 which is incompatible.
